# Prosessering av Energy Dispersive X-ray Spectroscopy data

Denne Jupyter Notebooken viser hvordan Energy Dispersive X-ray Spectroscopy (EDS) data kan analyseres. Spektroskopi er en veldig viktig datatype "klasse", som dukker opp i mange forskjellige teknikker.

### Målet med denne notebooken

- Dere skal kunne prosessere EDS dataene dere tok opp i SEM-laben
- Bli komfortable med å jobbe med spektroskopidata
- Lage figur som viser kjemisk komposisjon i dataene deres

### Notebook-planen

- Åpne datasettet, og utforske det
- Finne ut hvilke grunnstoffer som er i prøvematerialet
- Lage figur av dette

## Importere biblioteker

Først, plotte-biblioteket. Dette kan enten være `%matplotlib qt` for egne vinduer for plottene, eller `%matplotlib widget` for å få plottene i selve Jupyter Notebooken.

In [1]:
import matplotlib.pyplot as plt 
import numpy as np
import hyperspy.api as hs
from pprint import pprint

from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
import matplotlib.patheffects as patheffects

%matplotlib qt5

Så importere HyperSpy

## Åpne dataset

Dette gjøres via `hs.load`, som kan åpne en rekke dataformater, spesielt innenfor elektronmikroskopi. EDS datasettene fra SEM laben består av 4 filer: `.emsa`, `.txt`, `.raw` og `.rpl`.

Kjør de 2 cellene under for å i) åpne dataene, ii) kalibrere dem.

Dette lager et signal `s`.

In [2]:
# filename_rpl = "datasett/eds_window2 1.rpl"
# filename_emsa = "datasett/eds_window2.emsa"
# filename_txt = "datasett/eds_window2.txt"
filename_rpl = "our_data/EDS Data 2.rpl"
filename_emsa = "our_data/eds_regulaer.emsa"
filename_txt = "our_data/eds_regulaer.txt"

(Merk: det kan hende dere får en feilmelding i `np.loadtxt` på grunn av mu, hvis dette skjer så gå inn i tekstfilen og endre den til u.)

In [3]:
s = hs.load(filename_rpl).T
s.set_signal_type('EDS_SEM')
s_c = hs.load(filename_emsa, signal_type='EDS_SEM')
s.get_calibration_from(s_c)
width = float(np.loadtxt(filename_txt, skiprows=4, max_rows=1, dtype=object, usecols=2)[()][:-2])
scale = width / s.axes_manager.navigation_shape[0]
s.axes_manager.navigation_axes[0].scale = scale
s.axes_manager.navigation_axes[1].scale = scale
s.axes_manager.navigation_axes[0].units = "um"
s.axes_manager.navigation_axes[1].units = "um"
pprint(s)

<EDSSEMSpectrum, title: , dimensions: (1024, 704|2048)>


Så skriv `s` i cellen under, og kjør den

Her ser vi et par viktige ting: signalet er et `EDSSEMSpectrum`, og den har 3 dimensjoner! 
    
Dimensjonene ser vi helt i slutten, som har 3 tall (som nok vil være annerledes for deres datasett): `(1024, 704|2048)`. Tallene til venstre for `|` er navigasjonsdimensjonene, mens tallet til høyre for `|` er signaldimensjonen: `(NAVIGASJON 0, NAVIGASJON 1|SIGNAL 0)`.

I denne datatypen, så er de 2 navigasjonsdimensjonene probe-posisjonen, og signaldimensjonen er energien til røntgenstrålene.

Dette betyr at signaldimensjonen er 1-dimensjonal, som stemmer bra med at dette er spektroskopisk data.

## Enkel utforskning av dataene

Nå kan vi visualisere dataene, og se hvordan spektrumene ser ut. Bruk `plot` funksjonen i `s`, og utforsk datasettet.

Merk: bildene i Jupyter Notebooken kommer til å være forskjellig fra det dere får opp på deres egen datamaskin.

<img src="figurer/sem_eds_navigator.jpg" width=900 height=900 />

In [4]:
s.plot()

Her ser vi at hver probe-posisjon har veldig få røntgen-tellinger, ergo at signalet ikke er særlig bra. Noen steder kan vi se at det er klare topper, men disse er for det meste under 10 tellinger.

Dette skal vi gjøre noe med, men først vil vi utforske datasettet som funksjon av røntgenstråle-energien. Bruk transpose funksjonaliteten i `s`, og lag et nytt signal `st`: `st = s.T`

In [5]:
st = s.T
st

<Signal2D, title: , dimensions: (2048|1024, 704)>

Skriv `st` i cellen under, og kjør den.

Nå er signalet `Signal2D`: navigasjon- og signal-dimensjonene har blitt byttet om. Så nå kan vi navigere over datasettet som en funksjon av røntgen-energien, istedet for probe-posisjonen.

Så plot `st`. Merk at nå er navigatoren i "røntgen" plottet.

<img src="figurer/eds_transpose.jpg" width=900 height=900 />

Flytt navigatoren frem og tilbake, spesielt på de klare toppene, og se om forskjelliges steder på prøven lyser opp.

In [6]:
st.plot()

Selv her, så er tellingene veldig lave. Så la oss midle over flere probe-posisjoner og detektor-kanaler.

For dette, så bruker vi `rebin` funksjonen, som er i `st`. Bruk `scale` parameteren, og sett den til `(4, 8, 8)`, dette betyr at vi summerer 4 detektor-kanaler, 8 x-probe posisjoner, og 8 y-probe posisjoner. Bruk dette til å lage en ny variabel `st_rebin`.

Så kjør `st_rebin`, for å se dimensjonene til dette nye signalet.

In [6]:
st_rebin = st.rebin(scale=(4, 8, 8))
st_rebin

<Signal2D, title: , dimensions: (512|128, 88)>

Her ser vi at sammenlignet med det orginale `st` signalet, som var `(2048|1024, 704)`, så er `st_rebin` mye mindre. Spesifikt, at den nye størrelsen er `2048 / 4 | 1024 / 8, 704 / 8)`.

Deretter plot dette nye `st_rebin` signalet.

In [8]:
st_rebin.plot()

Nå er røntgen-tellingene mye høyere.

Så vi ser at det er noe interessant i dataene. Det neste steget er å lage bilder som viser hvor de forskjellige grunnstoffene er.

## Mer avansert

### Finne grunnstoffene

Det første vi må gjøre her, er å finne ut hvilke grunnstoffer vi har i prøvematerialet.

Enkleste måten å gjøre dette på, er å se på toppene vi har røntgen-signalet, kombinert med det vi vet om prøvematerialet.

Så: summer opp alle probeposisjonene, til et røntgen-energi signal. Bruk `sum` funksjonen i `s` til å lage et nytt signal `s_sum`.

In [7]:
s_sum = s.sum()

Så bruk `plot` funksjonen i `s_sum` til å visualisere dette signal, og finn ut hva slags grunnstoffer vi har.

<img src="figurer/eds_sum.jpg" width=900 height=900 />

Dere kan se hvilke topper de forskjellige grunnstoffene lager, ved for eksempel å bruke denne: https://www.jeolusa.com/DesktopModules/LiveContent/API/Image/Get?mid=4725&eid=1&Type=View&PortalId=2

In [8]:
s_sum.plot()

# Topper for eksempeldata [keV]:
# 0.84  Nikkel
# 1.74  Silisium
# 1.47  (Aluminium?)
# 0.72  Jern
# 0.38  Nitrogen
# 0.26  Karbon 
# 0.51  (Oksygen?)
# 0.017 (Beryllium ???? Liten peak)
# 7.46  Nikkel
# 6.39  Jern

# Topper for regulaer [keV]:
# 0.847 Nikkel
# 1.742 Silisium
# 0.72  Jern
# 0.38  Nitrogen
# 0.036 ????
# 1.10  (Gallium???? Vi har ikke deponert)
# 0.512 (Oksygen?)
# 0.26  (Karbon?)
# 1.477 (Aluminium?)

**Gå igjennom alle toppene, og prøv å finn ut hvilket grunnstoff de tilhører.**

Merk at samme grunnstoff kan ha flere topper, så hvis dere er usikre så er et triks å sjekke om de andre toppene også er med spektrumet.

### Rebinning av signalet

Som vi så tidligere, så er det litt få signaler i hver probe-posisjon. Så før vi begynner med den mer avanserte prosesseringen, så bruk `rebin` funksjonen i `s`, og bruk `scale` parameteren med `(8, 8, 1)` til å summere 8 x 8 probe-posisjoner. Bruk dette til å lage en ny variabel, `s_rebin`.

Hvis du er usikker på hvordan du gjør dette, husk at du kan få opp `docstring` for alle funksjoner i python ved å ha en `?` etter funksjonen. Så her, `s.rebin?`.

In [9]:
s_rebin = s.rebin(scale=(8, 8, 1))
s_rebin

<EDSSEMSpectrum, title: , dimensions: (128, 88|2048)>

### Legge til grunnstoffene i signalet

Nå som dere har funnet grunnstoffene, så må de legges til i `s_rebin` signalet.

Dette gjøres via `set_elements` funksjonen som er i `s_rebin`. Parameter som skal til `set_elements` må være en liste, og hvert grunnstoff må være i formen `"Si"`, `"Fe"`, ...

Tips: se i docstring til `set_elements` via "Shift + Tab" på tastaturet.

In [10]:
s_rebin.set_elements(["Ni", "Si", "Fe", "C", "Ga"])

Sjekk at alt har blitt lagt til, via `metadata.Sample` attribute i `s_rebin`

In [11]:
s_rebin.metadata

├── Acquisition_instrument
│   └── SEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 0.0
│       │       ├── elevation_angle = 35.0
│       │       ├── energy_resolution_MnKa = 130.0
│       │       ├── live_time = 7843.348632832
│       │       └── real_time = 8547.232421888
│       ├── Stage
│       │   └── tilt_alpha = 0.0
│       └── beam_energy = 5.0
├── General
│   ├── FileIO
│   │   └── 0
│   │       ├── hyperspy_version = 1.7.2
│   │       ├── io_plugin = hyperspy.io_plugins.ripple
│   │       ├── operation = load
│   │       └── timestamp = 2022-11-02T10:12:39.117822+01:00
│   ├── date = 
│   ├── original_filename = EDS Data 2.rpl
│   ├── time = 
│   └── title = 
├── Sample
│   └── elements = ['C', 'Fe', 'Ga', 'Ni', 'Si']
└── Signal
    └── signal_type = EDS_SEM

Så legger vi til røntgen linjene til disse grunnstoffene, via `add_lines` funksjonen i `s_rebin`

In [12]:
s_rebin.add_lines()

Så se hva som har blitt lagt til i metadataen, via `metadata.Sample` attribute i `s_rebin`

In [13]:
s_rebin.metadata

├── Acquisition_instrument
│   └── SEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 0.0
│       │       ├── elevation_angle = 35.0
│       │       ├── energy_resolution_MnKa = 130.0
│       │       ├── live_time = 7843.348632832
│       │       └── real_time = 8547.232421888
│       ├── Stage
│       │   └── tilt_alpha = 0.0
│       └── beam_energy = 5.0
├── General
│   ├── FileIO
│   │   └── 0
│   │       ├── hyperspy_version = 1.7.2
│   │       ├── io_plugin = hyperspy.io_plugins.ripple
│   │       ├── operation = load
│   │       └── timestamp = 2022-11-02T10:12:39.117822+01:00
│   ├── date = 
│   ├── original_filename = EDS Data 2.rpl
│   ├── time = 
│   └── title = 
├── Sample
│   ├── elements = ['C', 'Fe', 'Ga', 'Ni', 'Si']
│   └── xray_lines = ['C_Ka', 'Fe_La', 'Ga_La', 'Ni_La', 'Si_Ka']
└── Signal
    └── signal_type = EDS_SEM

Her ser dere at det bare har blitt lagt til en linje per grunnstoff. Dette er fordi den med lavest energi er det mest relevant.

Dette kan dere så plotte, ved å bruke `plot` med `xray_lines=True` argumentet

In [12]:
s_rebin.plot(xray_lines=True)

Så kan vi hente ut intensiteten for alle disse linjene, over hele datasettet.

Til dette bruker vi `get_lines_intensity` som er i `s_rebin`. Lagre resultatet til dette i en ny variabel: `linjer`. I tillegg, så bruk `plot_result=True` i `get_lines_intensity`. Dette vil åpne en plotte-vindu for hvert grunnstoff.

In [14]:
linjer = s_rebin.get_lines_intensity() #plot_result=True)

Så kan vi se hva som er i `linjer` variablen. Skriv `linjer` i cellen under, og kjør den.

In [15]:
linjer

[<BaseSignal, title: X-ray line intensity of : C_Ka at 0.28 keV, dimensions: (128, 88|)>,
 <BaseSignal, title: X-ray line intensity of : Fe_La at 0.70 keV, dimensions: (128, 88|)>,
 <BaseSignal, title: X-ray line intensity of : Ga_La at 1.10 keV, dimensions: (128, 88|)>,
 <BaseSignal, title: X-ray line intensity of : Ni_La at 0.85 keV, dimensions: (128, 88|)>,
 <BaseSignal, title: X-ray line intensity of : Si_Ka at 1.74 keV, dimensions: (128, 88|)>]

#### Sjekke integrasjonsvinduet

Her ser vi at den er en liste med signaler, et for hvert grunnstoff. For å plotte dem: `linjer[0].plot()`

Et mulig problem med denne typen prosessering, er hvis røntgen-linjene er så nærme at de overlapper. En måte å sjekke dette på, er å se hvor "bredde" integrasjonsvinduene er.

Dette gjøres enkelts med å først å summere datasettet igjen. Bruk `sum` via `s_rebin`, til å lage en ny variabel `s_sum2`.

In [12]:
linjer[0].plot()
s_sum2 = s_rebin.sum()

Så `plot` `s_sum2`, med argumentet `integration_windows='auto'`. Bruk forstørrelse-funksjonen til å sjekke at det ikke er for mye overlapp.

In [16]:
s_sum2.plot(integration_windows='auto')

### Lage bilder av hvor grunnstoffene er

Nå som vi kan se hvor de forskjellige grunnstoffene er, så lager vi en figur som viser dette.

Først henter vi ut et og et grunnstoff, kall disse `s_si`, `s_fe`, ... . Siden `linjer` er en liste, så gjøres dette med `linjer[0]`, `linjer[1]`, ... . Pass på å sjekke hvilket grunnstoff de forskjellige signalene er!

Viktig: disse signalene må transposes! Så kommandoen blir `linjer[0].T`.

In [16]:
s_car, s_fe, s_ga, s_ni, s_si = linjer
s_car, s_fe, s_ga, s_ni, s_si = s_car.T, s_fe.T, s_ga.T, s_ni.T, s_si.T
# s_ga.plot()

Så kan vi lage en matplotlib figur med et subplot per grunnstoffer dere har.

- Tips 1: har dere backscatter elektron eller Sekundærelektron bilde, så er det også fint å ta med! Da må dere ha et ekstra subplot i tillegg.
- Tips 2: hvis dere har veldig mange grunnstoff, så kan den være en fordel å ha 2 vertikale rader med subplot.
- Tips 3: bruk figsize parameteren til å lage figures større, slik at subplotene passer inn. F.eks. hvis dere har 4 horisontale subplot, så kan `figsize=(20, 5)` passe bra.

Importer matplotlib, lag figur og subplot objekter.

In [17]:
fontprops = fm.FontProperties(size=18)

scalebar_kwargs = {'size': 5, 'label': '5 um', 'loc': 4, 'frameon': False, 'color': 'white', 'size_vertical': 0.2, 'label_top': False, 'fontproperties': fontprops}
nice_effect = [patheffects.withStroke(linewidth=2, foreground='black', capstyle="round")]

def add_scalebar(ax: plt.Axes): 
    scalebar = AnchoredSizeBar(transform=ax.transData, **scalebar_kwargs)
    # Denne legger til et svart omriss rundt scalebar teksten, for å gjøre den lettere å lese
    scalebar.txt_label._text.set_path_effects(nice_effect)
    ax.add_artist(scalebar)

In [23]:
pprint(s_ni.axes_manager.signal_extent)

(0.048535156249999996, 14.13759765625, 0.048535156249999996, 9.70009765625)


In [30]:
fig, axs = plt.subplots(2, 2)
axs: list[plt.Axes] = axs.flat
for ax in axs:
    ax.axis("off")
    add_scalebar(ax)

cax_ni = axs[0].imshow(s_ni, cmap="Blues_r" ,extent=s_ni.axes_manager.signal_extent)
cax_ni.set_clim(17, 45)
axs[0].annotate("Ni", (0.05, 0.80), xycoords="axes fraction", fontsize=20, color="white", path_effects=nice_effect)
fig.colorbar(cax_ni, ax=axs[0], fraction=0.033, pad=0.04)

cax_si = axs[1].imshow(s_si, cmap="Reds_r", extent=s_si.axes_manager.signal_extent)
cax_si.set_clim(5, 20)
axs[1].annotate("Si", (0.05, 0.80), xycoords="axes fraction", fontsize=20, color="white", path_effects=nice_effect)
fig.colorbar(cax_si, ax=axs[1], fraction=0.033, pad=0.04)

cax_fe = axs[2].imshow(s_fe, cmap="Greens_r", extent=s_fe.axes_manager.signal_extent)
cax_fe.set_clim(3, 14)
axs[2].annotate("Fe", (0.05, 0.80), xycoords="axes fraction", fontsize=20, color="white", path_effects=nice_effect)
fig.colorbar(cax_fe, ax=axs[2], fraction=0.033, pad=0.04)

cax_ga = axs[3].imshow(s_ga, cmap="Purples_r", extent=s_ga.axes_manager.signal_extent)
cax_ga.set_clim(2, 12)
axs[3].annotate("Ga", (0.05, 0.80), xycoords="axes fraction", fontsize=20, color="white", path_effects=nice_effect)
fig.colorbar(cax_ga, ax=axs[3], fraction=0.033, pad=0.04)

fig.tight_layout()
# fig.savefig("regulaer_eds.png", dpi=300)

Så bruk `imshow` med forskjellige fargekart (`cmap`) til å visualisere dataene: for eksempel `"Blues_r"`, `"Greens_r"`, ... Se [matplotlib dokumentasjonen](https://matplotlib.org/stable/tutorials/colors/colormaps.html) for en fullstendig liste over alle fargekartene.

Et ekstra plotte-element som burde være med her er en ting som viser antall tellinger. Dette gjøres ved å:

- `cax_si = ax_si.imshow(....., extent=s_si.axes_manager.signal_extent, ....)` 

### Sette clim

I noen datasett, så kan det være vanskelig å se de interessante delene av dataene, fordi andre deler av datasettet entent har veldig høye eller veldig lave verdier.

Hvis dette er tilfellet, så bruk `cax_...` variablene til å sette "fargenivået" i plottene. Dette gjøres via `set_clim` funksjonen i (f.eks.) `cax_si`.

Merk at dere må finne ut hva de gode verdiene er her. En måte å gjøre det er å plotte dataene (f.eks. i `s_si`), flytte musepekeren over de interessante områdene, og se hva verdien er der (helt øverst til høyre `intensity`).

#### Legge til colorbars

I denne typen figurer, så er det fint å vise hvor mange tellinger man har. Dette gjøres via `colorbar`.

- `fig.colorbar(cax_si, ax=ax_si, label="Si")`

Merk: hvis dere får to colorbars på samme sublplot, så har dere kjørt `fig.colorbar(...)` to ganger på samme subplot. For å fikse dette, så lag figuren på nytt, fra `plt.subplots(...)`.

Så bruk tingene dere har lært i de andre dataøvingen, til å lage en figur:

- Legg til scalebar
- Ha annoteringer (a, b, c, ...), skriv hvilket grunnstoff det er: "Fe", "Si", ...
- Fjern tomrommet som kommer rundt plottene
- Fjern tallene som er rundt bildene. Her må dere mest sannsynlig manuelt stille på `figsize` til det blir bra

Eksempel på en sånn figur, men uten alle annoteringene

<img src="figurer/eds_kart.jpg" width=900 height=900 />